In [5]:
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
def get_index(nurse_index, day_index):
    return nurse_index * n_days + day_index

def get_nurse_and_day(index):
    nurse_index, day_index = divmod(index, n_days)
    return nurse_index, day_index

In [8]:
n_nurses = 3      # count nurses n = 1 ... n_nurses
n_days = 4      # count scheduling days as d = 1 ... n_days
size = n_days * n_nurses
valid_edges = []
cons_1_valid_edges = []
cons_2_valid_edges = []
cons_3_valid_edges = []
lagrange_hard_shift = 1.3
workforce = 1     # Workforce function W(d) - set to a constant for now
effort = 1 
G = nx.Graph()
a = 3.5
num_colors = 2
colors = range(num_colors)
#COns 3
lagrange_soft_nurse = 0.3      # Lagrange parameter for soft nurse, gamma
preference = 1                # preference function - constant for now
min_duty_days = int(n_days/n_nurses)

#Valid edges for Cons#1
for i in range(size):
    if (i+1)%n_days !=0:
        cons_1_valid_edges.append((i,i+1))
        if((i,i+1) not in valid_edges):
            valid_edges.append((i,i+1))
        
#Valid edges for Cons#2
for i in range(size):
    for j in range(i,size):
        if (j%(n_days)==(i%n_days)):
            cons_2_valid_edges.append((i,j))
            if((i,j) not in valid_edges):            
                valid_edges.append((i,j))
#Valid edges for Cons#3
for nurse in range(n_nurses):
    for day1 in range(n_days):
        for day2 in range(day1 + 1, n_days):
            ind1 = get_index(nurse, day1)
            ind2 = get_index(nurse, day2)
            
            cons_3_valid_edges.append((ind1,ind2))
            if((ind1,ind2) not in valid_edges): 
                valid_edges.append((ind1,ind2))
G.add_edges_from(valid_edges)



e_offset = (lagrange_hard_shift * n_days * workforce ** 2) + (lagrange_soft_nurse * n_nurses * min_duty_days ** 2)
dqm = DiscreteQuadraticModel()
# G.add_edges_from([(0, 1), (1, 2),(2, 3), (4, 5),(5, 6), (6, 7), (8, 9),(9, 10),(10, 11)]) #edges_for_4
# Parameters for hard nurse constraint
# a is a positive correlation coefficient for implementing the hard nurse
# constraint - value provided by Ikeda, Nakamura, Humble

# #Constraint1
for p in G.nodes:
    dqm.add_variable(num_colors, label=p)
for p in G.nodes:
    dqm.set_linear(p, colors)
for p0, p1 in G.edges:
    if(p0,p1) in cons_1_valid_edges:
        dqm.set_quadratic(p0, p1, {(c, c): a for c in colors})


#for cons#2
# Parameters for hard shift constraint
# Hard shift constraint: at least one nurse working every day
# Lagrange parameter, for hard shift constraint, on workforce and effort


linear_term = np.linspace(0,1,2)*(lagrange_hard_shift * (effort ** 2 - (2 * workforce * effort)))
for p in G.nodes:
    dqm.set_linear(p,linear_term)
for p0, p1 in G.edges:
    if(p0 != p1) and (p0,p1) in cons_2_valid_edges:
        dqm.set_quadratic(p0, p1, {(c, c): 2 * lagrange_hard_shift * effort ** 2  for c in colors})

#Cons 3
linear_term_cons3 = np.linspace(0,1,2)*(lagrange_soft_nurse * (preference ** 2 - (2 * min_duty_days * preference)))
for p in G.nodes:
    dqm.set_linear(p,linear_term_cons3)
for p0, p1 in G.edges:
    if(p0 != p1) and (p0,p1) in cons_3_valid_edges and (p0,p1) not in cons_1_valid_edges and (p0,p1) not in cons_2_valid_edges :
        dqm.set_quadratic(p0, p1, {(c, c): 2 * lagrange_soft_nurse * preference ** 2  for c in colors})
sampler = LeapHybridDQMSampler()
sampleset = sampler.sample_dqm(dqm)
sample = sampleset.first.sample
energy = sampleset.first.energy
print("Solution: ", sample)
print("Solution energy: ", energy)

#--------------------------------------------------------------#


Solution:  {0: 0, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 0, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0}
Solution energy:  12.199999999999996


In [7]:
# Graphics
sched = [get_nurse_and_day(j) for j in range(size) if sample[j] == 1]
str_header_for_output = " " * 11
str_header_for_output += "  ".join(map(str, range(n_days)))
print(str_header_for_output)
for n in range(n_nurses):
    str_row = ""
    for d in range(n_days):
        outcome = "X" if (n, d) in sched else " "
        if d > 9:
            outcome += " "
        str_row += "  " + outcome
    print("Nurse ", n, str_row)

           0  1  2  3  4  5  6  7  8  9  10
Nurse  0      X     X     X     X     X    
Nurse  1   X     X     X     X     X     X 
Nurse  2   X     X     X     X     X     X 
